# Automatic Political Stance Detection

A combination of topic modelling techniques and sentiment analysis applied to British newspaper articles written about the topic Brexit

![Pipeline](images/pipeline_ANLP_project.png)

In [1]:
%%html
<script>
  function code_toggle() {
    if (code_shown){
      $('div.input').hide('500');
      $('#toggleButton').val('Show Code')
    } else {
      $('div.input').show('500');
      $('#toggleButton').val('Hide Code')
    }
    code_shown = !code_shown
  }

  $( document ).ready(function(){
    code_shown=false;
    $('div.input').hide()
  });
</script>
<form action="javascript:code_toggle()"><input type="submit" id="toggleButton" value="Show Code"></form>

In [2]:
# All imports
import re, string, gensim, os
from nltk.tokenize import word_tokenize
from subprocess import Popen, PIPE, STDOUT
from nltk.corpus import stopwords
from gensim import corpora, models
from pprint import pprint
from sklearn import metrics
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.pyplot as plt
import numpy as np
from itertools import groupby
from nltk.sentiment.vader import SentimentIntensityAnalyzer

c:\users\tim\appdata\local\programs\python\python36\lib\site-packages\gensim\utils.py:1167: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
c:\users\tim\appdata\local\programs\python\python36\lib\site-packages\nltk\twitter\__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


## Preprocessing

To make our data suitable for use with the topic modelling techniques & sentiment analysis, we first extract the headlines and articles, using regular expressions. This removes all metadata that is included in the downloads from LexisNexis. We then tokenise and lemmatise the articles and remove stopwords.

### Cleaning

In [3]:
with open("sample_for_notebook.txt") as f_raw:
    print("Raw data\n")
    print(f_raw.read())

Raw data


                              1 of 1000 DOCUMENTS



                                  The Guardian

                     December 1, 2016 Thursday 8:18 PM GMT

It won't be easy to stop Brexit. But here are four ways to do it;
Chip away, every day. This is a long game but, as harsh reality bites, time will
be on the side of the remainers

BYLINE: Martin Kettle

SECTION: OPINION

LENGTH: 912 words


Those of us with only a smattering of knowledge about the ancient world know one
thing about Cato the Elder. During Rome's long wars against Hannibal, Cato ended
every speech in the senate with the same words: "Carthage must be destroyed."

" Brexit must be stopped" is unlikely to last as long as Cato's catchphrase has
managed to. But it focuses the mind. Those who think Brexit must be stopped are
not the majority. But they have a case and a cause, and they are right. So how
might stoppage be achieved?

Probably not by a political movement headed by Tony Blair. The former prime
mi

In [4]:
def extract_headlines(cleaned_article_list, output_file_hl):
    '''Extracts just the headline from the raw data and saves it to a text file. 
    Used in conjunction with the function clean_corpora.'''
    
    open(output_file_hl, 'w').close()
    
    with open(output_file_hl, "a") as output_hl:
        for cleaned_hl in cleaned_article_list:
            hl = re.sub(r"(BYLINE:.*|SECTION:.*|BODY: .*)$", "", cleaned_hl)
            print(hl.strip(), file=output_hl)
        
def clean_corpora(input_file, output_file, output_file_headlines=None, guardian=False, sun=False, telegraph=False, guardian_hl=False, 
                  sun_hl=False, telegraph_hl=False):
    '''Takes the raw LexisNexis files as input and extracts the headlines & articles and just headlines (optional), 
    writes them to a text file. Due to subtle formatting differences between the newspapers, 
    there are slight differences in the preprocessing'''
    
    open(output_file, 'w').close() #clears output file
    g_for_hl, s_for_hl, t_for_hl = [], [], []
    g_hl, t_hl, s_hl = [], [], []
    
    text = open(input_file).read()
    split_text = text.strip().split("All Rights Reserved")
    
    with open(output_file, "a") as output:
        for un_article in split_text:
            un_article1 = un_article.replace("\n", " ")
            article3 = un_article1.replace("\\", "")
            article2 = re.sub(r"(BYLINE: .* [0-9]* words|SECTION: .* LETTER|SECTION: .* words)", "", article3)
            article = re.sub(r"(LOAD-DATE:.*|LOAD-DATE)$", "", article2)
            g_for_hl.append(article3)
            s_for_hl.append(article3)
            
    
            if guardian:          
                article_g = re.sub(r"^(.*GMT)", "", article)
                print(article_g.strip(), file=output)
                
            if sun:
                article_s = re.sub(r"^(.*National Edition|.*Edition [0-9]*;)", "", article)
                article_s = re.sub(r"([a-zA-Z.]*@the[-]*sun\.co\.uk.*)$", "", article_s)
                print(article_s.strip(), file=output)
                
            if telegraph:
                article_t1 = re.sub(r"^(.*National Edition|.*Edition [0-9]*;|Scotland)", "", article)
                t_for_hl.append(article_t1)
                article_t = re.sub(r"(Copyright [0-9]+ Telegraph Media Group Limited|BYLINE: .* BODY:|[0-9]* of [0-9]* DOCUMENTS|HEADLINE:)", "", article_t1)
                print(article_t.strip(), file=output)
                
        if guardian_hl:
            for pre_hl in g_for_hl:
                article_g1 = re.sub(r"^(.*GMT)", "", pre_hl)
                g_hl.append(article_g1)
            extract_headlines(g_hl, output_file_headlines)

        if sun_hl:
            for pre_hl in s_for_hl:
                article_s1 = re.sub(r"^(.*National Edition|.*Edition [0-9]*;)", "", pre_hl)
                s_hl.append(article_s1)
            extract_headlines(s_hl, output_file_headlines)
   
        if telegraph_hl:
            for pre_hl in t_for_hl:
                article_t1 = re.sub(r"(HEADLINE:|Scotland)", "", pre_hl)
                t_hl.append(article_t1)
            extract_headlines(article_t1, output_file_headlines)

In [5]:
clean_corpora("sample_for_notebook.txt", "sample_output.txt", "sample_output_headlines.txt",
             guardian=True, guardian_hl=True)

with open("sample_output.txt") as f, open("sample_output_headlines.txt") as f_hl:
    print("\nHeadline and article\n")
    print(f.read())
    print("\nHeadline\n")
    print(f_hl.read())


Headline and article

It won't be easy to stop Brexit. But here are four ways to do it; Chip away, every day. This is a long game but, as harsh reality bites, time will be on the side of the remainers     Those of us with only a smattering of knowledge about the ancient world know one thing about Cato the Elder. During Rome's long wars against Hannibal, Cato ended every speech in the senate with the same words: "Carthage must be destroyed."  " Brexit must be stopped" is unlikely to last as long as Cato's catchphrase has managed to. But it focuses the mind. Those who think Brexit must be stopped are not the majority. But they have a case and a cause, and they are right. So how might stoppage be achieved?  Probably not by a political movement headed by Tony Blair. The former prime minister is not heading back into frontline politics. But he is one of the biggest names to insist that Brexit is not yet irrevocable. He told the New Statesman last week that Brexit " can be stopped if the Br

### Tokenization

In [6]:
# reads in the corpus, one headline/article per line
# returns an array of headlines/articles
def read_corpus(input_file):
	corpus = []
	with open(input_file) as f:
		for line in f.readlines():
			if line != '\n':
				corpus.append(line.strip())
	return corpus

# the tokenizer provided by nltk is used 
# returns a list of tokenized headlines/articles 
def tokenize_corpus(articles):
	tokenized_articles = []
	for sentence in articles:
		tokenized_articles.append(word_tokenize(sentence))
	return tokenized_articles

# Writes one tokenized article per line in a file
def write_to_file(tokenized_articles, output_file):
	with open(output_file, 'w') as o:
		for article in tokenized_articles:
			o.write(" ".join(article))
			o.write("\n")

In [7]:
sample_corpus = read_corpus("sample_output_headlines.txt")
tokenized_sample = tokenize_corpus(sample_corpus)
write_to_file(tokenized_sample, "tokenized_output_file.txt")
print(tokenized_sample)

[['It', 'wo', "n't", 'be', 'easy', 'to', 'stop', 'Brexit', '.', 'But', 'here', 'are', 'four', 'ways', 'to', 'do', 'it', ';', 'Chip', 'away', ',', 'every', 'day', '.', 'This', 'is', 'a', 'long', 'game', 'but', ',', 'as', 'harsh', 'reality', 'bites', ',', 'time', 'will', 'be', 'on', 'the', 'side', 'of', 'the', 'remainers']]


### Lemmatization

In [8]:
# reads in the corpus, one headline/article per line
# returns an array of headlines/articles
def get_sentences(input_file):
	sentences = []
	with open(input_file) as f:
		for line in f.readlines():
			if line != "\n":
				sentences.append(line.strip())
	return sentences

# the headlines/articles are lemmatized using the Treetagger
# returns a list of lists with the original word, the POS-Tag and the lemma per article
def annotate_articles(sentences):
	annotated_articles = []
	for sentence in sentences:
		sentence.lower()
		p = Popen(['/Applications/Treetagger/cmd/tree-tagger-english'], stdout=PIPE, stdin=PIPE, stderr=PIPE, encoding="utf8")
		out = p.communicate(input=sentence)[0]
		article = []
		annotated_words = out.split("\n")
		for word_anno in annotated_words:
			word_anno = word_anno.split("\t")
			if len(word_anno) == 3:
				annotation = (word_anno[0], (word_anno[1], word_anno[2]))
				article.append(annotation)
		annotated_articles.append(article)
	return annotated_articles

# for each word in an article the lemma is extracted
# if lemma is unknown, the original word is chosen
# return a list of lemmatized articles
def get_lemma(articles):
	lemma_articles = []
	for article in articles:
		lemma_article = []
		for word, anno in article:
			if anno[1] == "<unknown>":
				lemma_article.append(word)
			else:
				lemma_article.append(anno[1])
		lemma_articles.append(lemma_article)
	return lemma_articles

# Writes one lemmatized article per line in a file
def write_to_file(lemma_articles, out_file):
	with open(out_file, "w") as out:
		for article in lemma_articles:
			out.write(" ".join(article))
			out.write("\n")

In [9]:
lemmatized_sentences = get_sentences("tokenized_output_file.txt")
annotated_sample = annotate_articles(lemmatized_sentences)
lemmatised_sample = get_lemma(annotated_sample)
write_to_file(lemmatised_sample, "lemmatised_output_file.txt")
print(lemmatised_sample)

FileNotFoundError: [WinError 2] Das System kann die angegebene Datei nicht finden

### Stop Word Removal

In [ ]:
def remove_stop_words(processed_file, output_file):
    '''Takes the lemmatized, tokenized file as input and removes punctuation, stopwords and other strangely formatted
    words/punctuation'''
    
    stop_words_nltk = list(stopwords.words('english'))
    to_delete = ["brexit", "``", "''", "'s", "·", "wo", "n't", "...", "@card@"]
    
    stop_words = stop_words_nltk + to_delete + list(string.punctuation)

    with open(output_file, "w") as output:
        to_be_filtered = open(processed_file).readlines()
        corpus_list = [line.strip() for line in to_be_filtered]
        corpus_nlist = [each_word.split() for each_word in corpus_list]
        for sentence in range(0, len(corpus_nlist)):
            for word in corpus_nlist[sentence]:
                word = re.sub(r"^(\')", "", word) #getting rid of the ' at the beginning of some words
                if word.lower() not in stop_words:
                    output.write(word)
                    output.write(" ")
            output.write("\n") #line break after each article/hl

In [ ]:
remove_stop_words("lemmatised_output_file.txt", "filtered_output_file.txt")
with open("filtered_output_file.txt") as filtered:
    print(filtered.read())

## LDA

To get an interpretation of the content of the newspaper articles, we apply topic modeling to them. 
We use a Latent Dirichlet Allocation (LDA) model from the package gensim. 
This model assumes that our articles were written using a "generative process" in which every word of the article is created out of a distribution of words within a topic. 
In a first step, this distribution is created by pre-defining a number of topics and then building a model over the whole corpus. In a second step, the model is applied to the articles to find the specific topic mixture which best represents the article. These topic mixtures make up our feature vectors for the clustering.

In [10]:

class LDA():
    def __init__(self, filename1, filename2, stopwords, num_topics=10, no_below=20, no_above=0.5):
        self.num_topics = num_topics
        self.corpus = self.createCorpus(filename1, filename2, stopwords)
        self.model_corpus, self.dictionary = self.createModelCorpus(no_below, no_above)
        try:
            self.model = gensim.models.LdaModel.load("LDA_Models/ldamodel_topics="+str(num_topics)+"_no_above="+str(no_above))
        except:
            self.model = self.train_lda()
            self.model.save("LDA_Models/ldamodel_topics="+str(num_topics)+"_no_above="+str(no_above))

        self.corpus_feature_vectors = self.apply_lda()
        self.final_output = self.createFinalOutput()

    def createCorpus(self, filename1, filename2, stopwords):
        with open(filename1,encoding="utf-8") as f1:
            with open(filename2, encoding="utf-8") as f2:
                all_articles = f1.readlines()
                all_articles.extend(f2.readlines())
                corpus = [[token for token in article.strip().split(" ") if token not in stopwords] for article in all_articles]
        return corpus

    def createModelCorpus(self, no_below, no_above):
        dictionary = corpora.Dictionary(self.corpus)
        dictionary.filter_extremes(no_below=no_below, no_above=no_above)
        model_corpus = [dictionary.doc2bow(token) for token in self.corpus]
        return model_corpus, dictionary

    def train_lda(self):
        '''applies LDA to tokenized articles in a txt-file where each article is
        separated by a newsline'''
        ldamodel = gensim.models.ldamulticore.LdaMulticore(self.model_corpus, num_topics=self.num_topics, id2word=self.dictionary, passes=10)
        return ldamodel

    def apply_lda(self):
        corpus_feature_vectors = []
        for article in self.model_corpus:
            corpus_feature_vectors.append(self.model[article])
        return corpus_feature_vectors

    def createFinalOutput(self):
        output = []
        for vec in self.corpus_feature_vectors:
            vec_dic = dict(vec)
            output.append([(vec_dic[topic] if topic in vec_dic else 0) for topic in range(10)])
        return output

    def get_topics(self,num_topics=10,num_words=10,probs=False,formatted=False):
        #return topics without probabilites
        topics = self.model.show_topics(num_topics=num_topics,num_words=num_words,formatted=formatted)
        #print(topics)
        if not(probs):
            all_topics = []
            for topic in topics:
                all_words = []
                for word in topic[1]:
                    all_words.append(word[0])
                all_topics.append(all_words)
                #print(all_words)

            return(all_topics)
        else:
            return topics


### Applying LDA

Applying LDA and having a look at the 10 topics with the 5 most important words.

In [11]:
filename1 = "Corpora/filtered/filteredjust_hl_article_guardian_lemmatized.txt"
filename2 = "Corpora/filtered/filteredhl_article_tele_lemmatized.txt"

stopword = stopwords.words("english") + list(string.punctuation) + ["Mr","Mrs","BST","block-time","published-time"]

lda = LDA(filename1, filename2,stopwords=stopword, num_topics=10, no_below=20, no_above=0.7)

corpus_feature_vectors = lda.corpus_feature_vectors
output = lda.final_output
lda.get_topics(num_words=5,probs=True,formatted=True)

[(0,
  '0.022*"MP" + 0.017*"vote" + 0.011*"bill" + 0.010*"parliament" + 0.009*"Labour"'),
 (1,
  '0.024*"May" + 0.012*"Scotland" + 0.010*"deal" + 0.009*"Scottish" + 0.009*"referendum"'),
 (2,
  '0.024*"Labour" + 0.016*"party" + 0.012*"vote" + 0.009*"people" + 0.008*"Corbyn"'),
 (3,
  '0.014*"talk" + 0.012*"deal" + 0.011*"May" + 0.010*"European" + 0.008*"British"'),
 (4,
  '0.012*"deal" + 0.011*"Ireland" + 0.010*"Davis" + 0.009*"right" + 0.009*"citizen"'),
 (5,
  '0.012*"economy" + 0.009*"growth" + 0.009*"vote" + 0.009*"year" + 0.007*"referendum"'),
 (6,
  '0.011*"May" + 0.010*"Johnson" + 0.009*"Hammond" + 0.009*"deal" + 0.008*"minister"'),
 (7,
  '0.013*"trade" + 0.011*"Ireland" + 0.008*"deal" + 0.008*"European" + 0.007*"country"'),
 (8,
  '0.007*"people" + 0.007*"European" + 0.007*"Trump" + 0.007*"vote" + 0.006*"year"'),
 (9,
  '0.012*"financial" + 0.011*"business" + 0.010*"London" + 0.009*"market" + 0.009*"bank"')]

### K-Means and Visualisation
The feature vectors consist of 12 values which are the proportions in which each of the 12 topics is represented in the article. With these we can imagine every article being located somewhere in the 12-dimensional topic space. By using k-means clustering we would like to find different clusters among the articles. Being in one cluster means that a group of articles make use of a similiar topic mix. We evaluate the quality of the cluster by the "elbow" method using the sum of squared intra-cluster distance of the articles. We plot histograms for each cluster showing what proportions of either Guardian or Telegraph articles make up the cluster.

In [14]:
class kmeans():

    def __init__(self,feature_vector,image_dir,r_state=42):

        self.feature_vector = feature_vector
        self.r_state = r_state
        self.dir = "cluster_images/"+image_dir+"/"
        guardian = np.ones(998)
        sun = np.zeros(876)
        self.both = np.concatenate([guardian, sun])
        self.centroids=None


    def cluster(self,k):
        '''
        partitions the data into k clusters using the k_means algorithm
        '''
        clustering = KMeans(n_clusters=k,random_state=self.r_state)
        labels = clustering.fit_predict(self.feature_vector)
        self.labels = labels

        self.centroids = clustering.cluster_centers_
        return(labels,clustering.inertia_)

    def visualize_data(self,fname):
        '''
        shows a 2-dimensional scatter plot of data with each color representing the cluster and the
        form of the data point (triange/circle) indicates from which newspaper the article is
        '''
        X_reduced = PCA(n_components=2).fit_transform(self.feature_vector)
        fig = plt.figure()

        ax = fig.add_subplot(111)
        ax.scatter(X_reduced[:, 0], X_reduced[:, 1], c=self.labels,marker="^", s=self.both*10, edgecolor="red", linewidth=0.3)
        ax.scatter(X_reduced[:,0], X_reduced[:,1], c=self.labels,marker="o", s=((self.both-1)*-1)*10, edgecolor="black", linewidth=0.3)
        ax.set_xlabel('x')
        ax.set_ylabel('y')
        #plt.colorbar(scatter)

        if not os.path.exists(self.dir):
            os.makedirs(self.dir)
        plt.savefig(self.dir+fname,dpi=600)
        plt.close()

    def plot_elbow(self,k_range,fname,individ_plot=False):
        '''
        plots the intra-cluster distance of all clustr against different number of cluster. Needs the range of k.
        '''
        distorsions = []
        for k in k_range:
            labels, score = self.cluster(k)
            distorsions.append(score)
            if individ_plot:
                self.visualize_data("cluster_"+str(k))

        plt.figure(figsize=(15, 5))
        plt.xlabel("number of clusters: k")
        plt.ylabel("Sum of squared distances of samples to their closest cluster center.")
        plt.plot(k_range, distorsions)
        plt.grid(True)
        plt.savefig(self.dir+fname,dpi=600)
        plt.close()

    def plot_histogram2(self, fname, k):
        '''
        plots the histogram for the distribution of articles in each cluster
        '''
        grouped = [[] for x in range(k)]
        hist = [[] for x in range(k)]
        # the histogram of the data
        verteilung = list(zip(self.both, self.labels))
        verteilung = sorted(verteilung, key=lambda x: x[1])

        for key, group in groupby(verteilung, lambda x: x[1]):
            for thing in group:
                #print(key, thing)
                grouped[key].append(int(thing[0]))
            hist[key].append(np.histogram(grouped[key],bins=2)[0])

        guardian = []
        telegraph = []
        for cluster in hist:
            guardian.append(cluster[0][0])
            telegraph.append(cluster[0][1])

        ind = np.arange(k)  # the x locations for the groups
        width = 0.35  # the width of the bars

        fig, ax = plt.subplots()
        rects1 = ax.bar(ind, guardian, width, color='b')
        rects2 = ax.bar(ind + width, telegraph, width, color='y')

        # add some text for labels, title and axes ticks
        ax.set_ylabel('Frequency in topic clusters')
        ax.set_title('Distribution of Guardian/Telegraph articles in each cluster')
        ax.set_xticks(ind + width / 2)
        ax.set_xticklabels(range(k))

        ax.legend((rects1[0], rects2[0]), ('The Guardian', 'Telegraph'))

        print(str(self.dir + fname))
        plt.savefig(str(self.dir + fname))
        plt.clf()


    # print(stopword)
# filename1 = "Corpora/filtered/filteredjust_hl_article_guardian_lemmatized.txt"
# filename2 = "Corpora/filtered/filteredhl_article_tele_lemmatized.txt"

#     # no_below : No words which appear in less than X articles
#     # no_above : No words which appear in more than X % of the articles
num_topics = 10
no_above = 0.7
# lda = LDA(filename1, filename2, stopword,num_topics=num_topics,no_below=20, no_above=no_above)
# corpus_feature_vectors = lda.corpus_feature_vectors
# output = lda.final_output

k_means = kmeans(output, "num_topics="+str(num_topics)+"_no_above="+str(no_above).replace(".",""))

#print(k_means.centroids)
k_means.plot_elbow(range(4,17,2), "Elbow plot", individ_plot=True)



after the model was applied, a kink in the elbow plot shows where a "good" number of clusters is reached. While we used 10 topics because this agreed with our manually annotated texts, the clustering reveals that there are actually 8 clusters which represent the same micture of topics. The Elbow plot looks as follows:
![Pipeline](cluster_images\num_topics=10_no_above=07\Elbow plot.png)

PCA calculates the 2 dimension with the highest explained variance and projects these dimensions of the cluster only: 
![Pipeline](cluster_images\num_topics=10_no_above=07\cluster_8.png)

### Histograms

In [16]:
with open("cluster_images/top10topics.csv", "a") as top:
    top.write("num_topics=" + str(num_topics) + "_no_above=" + str(no_above).replace(".", "") + ",")
    topics = lda.get_topics(num_words=5)
    for topic_words in topics:
        top.write(str(topic_words) + ",")
        top.write("\n")
    for k in [8]:
        k_means = kmeans(output, "histograms")
        labels,score = k_means.cluster(k)
        k_means.plot_histogram2("num_topics="+str(num_topics)+"_no_above="+str(no_above).replace(".","")+"_k="+str(k),k)



cluster_images/histograms/num_topics=10_no_above=07_k=8


Plotting a histogram showing the distribution of Guardian and Telegraph articles in each cluster yields: 
![Pipeline](cluster_images\histograms\num_topics=10_no_above=07_k=8.png)

### Investigating the cluster and their centroids:

![Pipeline](images\political_stance_sample_interpretation.png)

Lets investigate the clusters a bit. An exemplary interpretation for the first three cluster can be given by:

If we have a closer look on Cluster 2 we can see that The Guardian published more articles than The Telegraph. The most frequent topic in Cluster 2 is Topic 2, which consisting words tell us most about the UK inner conflict with Scottish ambitions for independence. Since the SNP is against Brexit a news coverage of The Guardian is more likely.

Looking at Cluster 1 we can observe a focus on the Tory party debates. Telling us, that The Telegraph is more publishing about the Conservative party.

In Cluster 3 The Guardian is again the dominant newspaper, publishing more about the British-European negotiations.

## Sentiment Analysis


If articles present the same topics, the manner the topics are presented can still differ. Either a negative, positive or neutral sentiment can be expressed towards a topic. To be able to determine whether that is a the case we apply the Vader sentiment analysis system (Hutto & Gilbert, 2014) implemented in the nltk module to the headlines of the articles. We don't expect the system to work well if applied to the whole article, as many different sentiments are expressed in a text, the score would probably be balanced. Vader is a rule-based sentiment analysis system that was originally applied to user generated content. As headlines are only short texts we expected it to work well with headlines.

In [ ]:
snt = SentimentIntensityAnalyzer()

# the nltk vader sentiment analysis system is used to
# get the polarities of the headline sentences
# returns a list of list with the overall, negative, neutral and positive 
# value per headline
def get_sentiment_score(sentences):
	scores = []
	for sentence in sentences:
		score = snt.polarity_scores(sentence)
		#score_values = [ val for key, val in score.items] not right order
		score_values = []
		score_values.append(str(score["compound"]))
		score_values.append(str(score["neg"]))
		score_values.append(str(score["neu"]))
		score_values.append(str(score["pos"]))
		scores.append(score_values)
	return scores

# the scores are written to the output file 
def write_scores_to_file(out_file, scores):
	with open(out_file, "w") as o:
		o.write("Overall score\tNegative\tNeutral\tPositive\n")
		for score in scores:
			o.write("\t".join(score))
			o.write("\n")

In [ ]:
lemmatised_sentences = read_corpus("lemmatised_output_file.txt")
scores = get_sentiment_score(lemmatised_sentences)
print(scores)

### Comparison of manual and automatic score

We manually assigned scores to 100 headlines each from The Guardian and The Telegraph. The manual and the automatic scores are compared in two manners: 1. The polarity itself is compared, it is determined whether it lays in a predefined tolerance range. 2. It is determined whether the scores have the same polarity i.e. being negative (<0), positive (>0) or neutral (=0). As the system did not perform very well - only half of the headlines had the same polarity and only very few matched within the tolerance range - only few topic words could be found in the headlines - topic words only appeared in half of the headlines at all - we did not apply sentiment analysis to further interpret the clustered topics and we also did not improve the system to an aspect-based level.

In [ ]:
# only is the overall score is taken for the
# comparison
# it's rounded to one decimal place
# return a list of overall scores
def get_overall_score(scored_file):
	overall_scores = []
	with open(scored_file, 'r') as f:
		for line in f.readlines():
			if line != "\n":
				overall_scores.append(round(float(line.split("\t")[0]), 1))
	return overall_scores

# the manual scores are one score per line
# returns a list of manual scores
def read_manual_sa_score(manual_sa_file):
	manual_scores = []
	with open(manual_sa_file, 'r') as f:
		for line in f.readlines():
			if line != "\n":
				manual_scores.append(float(line.strip()))
	return manual_scores

# compares the automatic and manual scores
# the ok_range is a tolerance range of how much the scores may differ
# return the amount of right and wrong classified sentences
# and the indices of the wrongly classified sentences
def compare_scores(automatic, manual, ok_range):
	right = 0
	wrong = 0
	wrong_indices = []
	ok_range = float(ok_range)
	for i, (m_score, a_score) in enumerate(zip(automatic, manual)):
		if abs(m_score - a_score) <= ok_range: #and (m_score - a_score) >= 0.0:
			right += 1
		else:
			wrong += 1
			wrong_indices.append(i)
	return right, wrong, wrong_indices

# compares the automatic in manual scores in terms of polarity
# if both score are below, above or equal to 0 the have the same 
# polarity
# returns the amount of headlines with same and different polarity 
# and the indices of wrongly classified sentences
def get_score_neg_pos_neut(automatic, manual):
	same_pol = 0
	diff_pol = 0
	wrong_indices = []
	for i, (m_score, a_score) in enumerate(zip(automatic, manual)):
		if (m_score > 0 and a_score > 0) or  (m_score < 0 and a_score < 0) or (m_score == 0 and a_score == 0):
			same_pol += 1
		else:
			diff_pol += 1
			wrong_indices.append(i)
	return same_pol, diff_pol, wrong_indices

### Extract topic words from headlines

In [ ]:
# reads the headlines
# returns a list of words per headline
def get_headlines(article):
	articles = []
	with open(article) as art:
		for line in art.readlines():
			articles.append(line.strip().split())
	return articles

# extracts the most important words per topic
# returns a list of the words per topic
def get_topic_words(topic_array):
	topics = []
	with open(topic_array) as t:
		for line in t.readlines():
			line = line.split("[")
			topic_words = []
			for topic in line:
				topic = topic.replace("\'", "")
				topic = topic.replace("]", "")
				topic = topic.replace(",", "")
				topic = topic.split()
				topics.append(topic)
	return(topics[1:])

# reads the topic dist per article 
# returns a list of indices for the important topics  
def get_article_topic_dist(t_dist):
	most_imp = []
	with open(t_dist) as t:
		for line in t.readlines():
			line = line.strip().split()
			most_imp.append(line)
	return most_imp

# extracts the words that form the important topics 
# returns a list of important words per headline
def get_imp_words(topics, topic_dist):
	most_imp_words = []
	for dist in topic_dist:
		most_imp_per_art = []
		for topic_ind in dist:
			most_imp_per_art.extend(topics[int(topic_ind)])
		most_imp_words.append(most_imp_per_art)
	return most_imp_words

# extract the words that are in the headline and topic words
# returns a list of these words per article
def get_imp_headline_words(most_imp_words, headlines):
	words_in_headline = []
	for words, headline in zip(most_imp_words, headlines):
		headline_words = []
		for word in words:
			if word in headline:
				headline_words.append(word)
		words_in_headline.append(headline_words)
	return(words_in_headline)

# prints the percentage of headlines in which topic words exist
def get_percentage_imp_word_headline(most_imp_words, words_in_headline):
	percentages = []
	for imp_w, w_in_h in zip(most_imp_words, words_in_headline):
		if len(w_in_h) != 0:
			percentage = 100/ (float(len(imp_w)) / float(len(w_in_h)))
		else:
			percentage = 0
		percentages.append(percentage)
	articles_with_imp_words_in_headlines = 0
	ten_percent_in_headline = 0
	for per in percentages:
		if per > 0.0:
			articles_with_imp_words_in_headlines += 1
		if per >= 10.0:
			ten_percent_in_headline += 1
	print("{} articles have topic words in the headlines".format(articles_with_imp_words_in_headlines))
	print("{} articles have more than 10% of the topics words in their headline".format(ten_percent_in_headline))

In [ ]:
topic_words = get_topic_words("cluster_images/top10topicswith10wordswithoutprobs.csv")
topic_dist = get_article_topic_dist("topic_words_headline/tele_topic_dist_02.txt")
most_imp_words = get_imp_words(topic_words, topic_dist)
headlines = get_headlines("Corpora/lemmatized/hl_tele_lemmatized.txt")
words_in_headline = get_imp_headline_words(most_imp_words, headlines)
get_percentage_imp_word_headline(most_imp_words, words_in_headline)